In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
# setting the device
device = torch.device('cuda' if torch.cuda.is_available() else cpu)

In [ ]:
# Hyperparameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [ ]:
# setting the transforms:
transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.1302,), (0.3081, ))
    ]
)

In [ ]:
# Getting the dataset

train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=transform,
    download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=transform
)

In [ ]:
# loading the data

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
# Creating the model class

class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.input_size = input_size
    self.layer1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.output_layer = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.layer1(x)
    out = self.relu(out)
    out = self.output_layer(out)
    return out

In [ ]:
# Model, loss and optimizer 

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# training the model
total_steps = len(train_loader)

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):

    # reshaping to (, 748)
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # printing information
    if (i+1) % 100 == 0:
      print(f"Epoch: [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss [{loss.item():.4f}]")

Epoch: [1/2], Step [100/600], Loss [0.2711]
Epoch: [1/2], Step [200/600], Loss [0.2126]
Epoch: [1/2], Step [300/600], Loss [0.2675]
Epoch: [1/2], Step [400/600], Loss [0.1052]
Epoch: [1/2], Step [500/600], Loss [0.1671]
Epoch: [1/2], Step [600/600], Loss [0.1282]
Epoch: [2/2], Step [100/600], Loss [0.0770]
Epoch: [2/2], Step [200/600], Loss [0.0897]
Epoch: [2/2], Step [300/600], Loss [0.0938]
Epoch: [2/2], Step [400/600], Loss [0.0743]
Epoch: [2/2], Step [500/600], Loss [0.0767]
Epoch: [2/2], Step [600/600], Loss [0.1558]


In [ ]:
# Testing the model

n_correct = 0
n_samples = 0
model.eval()

with torch.no_grad():
  for images, labels in test_loader:
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)

    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

  acc = 100 * n_correct/n_samples
  print(f"Accuray of the model: {acc:.4f}")

Accuray of the model: 97.5000


In [ ]:
# saving the model
torch.save(model.state_dict(), "digit_classifier.pth")